# ANCE with Pyterrier  -  FUNKTIONIERT NICHT!

## Setup

- install libraries (if necessary)
- all imports here
- connecting to tira & printoptions etc.

In [ ]:
#!pip install transformers faiss-gpu faiss-cpu torch
#!pip install tira ir-datasets python-terrier
#!pip install sentence-transformers
!apt install libomp-dev
!pip install faiss

In [ ]:
!pip install --upgrade git+https://github.com/seanmacavaney/pyterrier_ance.git@reranker

In [ ]:
# ColBERTv2 checkpoint trained on MS MARCO Passage Ranking (388MB compressed)
!wget https://webdatamltrainingdiag842.blob.core.windows.net/semistructstore/OpenSource/Passage_ANCE_FirstP_Checkpoint.zip
!unzip Passage_ANCE_FirstP_Checkpoint.zip

In [ ]:
import os
import sys
import time
import json
import re
import importlib
import random

import numpy as np
import pandas as pd
import torch
import faiss
import pyterrier as pt
import pyterrier_ance

# Tira and Pyterrier Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.third_party_integrations import ir_datasets
from tira.rest_api_client import Client

In [ ]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

# Print options for pandas
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.precision", 4)
pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', '{:.5f}'.format)

# Use GPU if available
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"device: {device}")

# TODO: set seed!!

COLAB='google.colab' in sys.modules
if COLAB:
    # mount to drive
    pass


## The Index

In [ ]:
dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [ ]:
checkpoint="Passage ANCE(FirstP) Checkpoint"

#!rm -rf /anceindex

indexer = pyterrier_ance.ANCEIndexer(checkpoint_path=checkpoint, index_path="ance_index")#, num_docs=126958)
indexer.index(dataset.get_corpus_iter())

In [ ]:
del(indexer) # not needed -> free memory

## Retriever

In [ ]:
ance_retr = pyterrier_ance.ANCERetrieval(checkpoint_path="/content/Passage ANCE(FirstP) Checkpoint", index_path="/content/anceindex")

In [ ]:
bm25 = pt.BatchRetrieve(dataset.get_index(), wmodel="BM25")
ance_rerank = (bm25 % 100) >> pt.text.get_text(dataset, "text") >> pyterrier_ance.ANCETextScorer(checkpoint_path="/content/Passage ANCE(FirstP) Checkpoint")

## Evaluation

In [ ]:
# baselines from TIRA
bm25_baseline = tira.pt.from_submission('ir-benchmarks/tira-ir-starter/BM25 (tira-ir-starter-pyterrier)', dataset)
sparse_cross_encoder = tira.pt.from_submission('ir-benchmarks/fschlatt/sparse-cross-encoder-4-512', dataset)
rank_zephyr = tira.pt.from_submission('workshop-on-open-web-search/fschlatt/rank-zephyr', dataset)

In [ ]:
pt.Experiment(
    [bm25_baseline, sparse_cross_encoder, rank_zephyr, ance_retr, ance_rerank],
    dataset.get_topics(),
    dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "map"],
    names=["BM 25 (Baseline)", "Sparse Cross Encoder", "RankZephyr", "Ance", "Ance Rerank"]
)